# Day12_3: 첫 ML 모델 (First ML Model) - 정답

## 🎯 실습 퀴즈 정답

---

In [ ]:
# 환경 설정
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

import warnings
warnings.filterwarnings('ignore')

# 데이터 로드
titanic = sns.load_dataset('titanic')
print("라이브러리 및 데이터 로드 완료!")

---

### Q1. 데이터 탐색 ⭐

**문제**: titanic 데이터에서 생존자(survived=1) 수와 사망자(survived=0) 수를 각각 출력하세요.

In [ ]:
# 정답 코드
survival_counts = titanic['survived'].value_counts()
print(survival_counts)

In [ ]:
# 테스트
assert survival_counts[0] == 549, "사망자 수가 549가 아닙니다"
assert survival_counts[1] == 342, "생존자 수가 342가 아닙니다"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `value_counts()` 메소드는 각 고유값의 빈도를 계산합니다.
- 기본적으로 내림차순 정렬됩니다.

**핵심 개념**:
- `value_counts()`: 범주형 변수의 분포 확인에 필수
- 불균형 데이터 확인: 생존(38%) vs 사망(62%)

**대안**:
```python
# 방법 2: groupby 사용
titanic.groupby('survived').size()

# 방법 3: 직접 계산
print(f"사망: {(titanic['survived'] == 0).sum()}")
print(f"생존: {(titanic['survived'] == 1).sum()}")
```

**흔한 실수**:
- `count()` 사용 (결측치 제외됨)
- 컬럼명 오타 ('survive' vs 'survived')

**실무 팁**:
- 타겟 변수의 불균형 비율은 모델 성능에 영향을 줍니다.
- 심한 불균형(90:10 이상)은 별도 처리 필요

---

### Q2. 그룹별 생존율 ⭐

**문제**: 객실 등급(pclass)별 평균 생존율을 계산하세요.

In [ ]:
# 정답 코드
pclass_survival = titanic.groupby('pclass')['survived'].mean()
print("객실 등급별 생존율:")
print(pclass_survival)

In [ ]:
# 테스트
assert len(pclass_survival) == 3, "3개 등급이 있어야 합니다"
assert pclass_survival[1] > pclass_survival[3], "1등석 생존율이 3등석보다 높아야 합니다"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `groupby('pclass')`: 객실 등급별로 그룹화
- `['survived'].mean()`: 생존 컬럼의 평균 (0과 1의 평균 = 생존율)

**핵심 개념**:
- 이진 변수(0/1)의 평균 = 1의 비율 = 생존율
- 1등석(63%) > 2등석(47%) > 3등석(24%)

**대안**:
```python
# 백분율로 표시
(titanic.groupby('pclass')['survived'].mean() * 100).round(1)

# 생존자 수와 함께
titanic.groupby('pclass')['survived'].agg(['mean', 'sum', 'count'])
```

**흔한 실수**:
- `sum()` 사용 (비율이 아닌 절대값)
- 그룹화 없이 전체 평균 계산

**실무 팁**:
- 그룹별 타겟 비율 분석은 특성의 예측력을 파악하는 첫 단계입니다.
- 차이가 클수록 예측에 유용한 특성

---

### Q3. 결측치 처리 ⭐⭐

**문제**: age 컬럼의 결측치를 평균값으로 대체하고, 결측치가 모두 처리되었는지 확인하세요.

In [ ]:
# 정답 코드
# 원본 보존을 위해 복사
df = titanic.copy()

# 결측치 처리 전
print(f"처리 전 결측치: {df['age'].isnull().sum()}개")

# 평균값으로 대체
df['age'] = df['age'].fillna(df['age'].mean())

# 결측치 처리 후
print(f"처리 후 결측치: {df['age'].isnull().sum()}개")

In [ ]:
# 테스트
assert df['age'].isnull().sum() == 0, "결측치가 남아있습니다"
assert len(df) == len(titanic), "데이터 크기가 변경되면 안됩니다"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. `isnull().sum()`으로 결측치 수 확인
2. `mean()`으로 평균 계산 (결측치 제외)
3. `fillna()`로 결측치를 평균값으로 대체

**핵심 개념**:
- `fillna()`: 결측치를 지정 값으로 대체
- 평균 vs 중앙값: 이상치가 많으면 중앙값 권장

**대안**:
```python
# 중앙값 대체 (이상치에 강건)
df['age'] = df['age'].fillna(df['age'].median())

# 그룹별 평균 (더 정교)
df['age'] = df.groupby('pclass')['age'].transform(
    lambda x: x.fillna(x.mean())
)
```

**흔한 실수**:
- 원본 데이터 직접 수정 (copy() 없이)
- 전체 평균이 아닌 대체 후 평균으로 계산

**실무 팁**:
- 결측치 비율이 5% 미만: 삭제 고려
- 5~30%: 대체(imputation) 권장
- 30% 이상: 해당 컬럼 삭제 또는 결측 여부를 새 특성으로

---

### Q4. 범주형 인코딩 ⭐⭐

**문제**: sex 컬럼을 숫자로 인코딩하세요 (male=0, female=1).

In [ ]:
# 정답 코드
df = titanic.copy()

# 인코딩 전
print("인코딩 전:")
print(df['sex'].value_counts())

# map을 사용한 인코딩
df['sex'] = df['sex'].map({'male': 0, 'female': 1})

# 인코딩 후
print("\n인코딩 후:")
print(df['sex'].value_counts())

In [ ]:
# 테스트
assert df['sex'].dtype in ['int64', 'int32', 'float64'], "숫자형이어야 합니다"
assert set(df['sex'].unique()) == {0, 1}, "0과 1만 있어야 합니다"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `map()` 메소드에 딕셔너리를 전달하여 값 변환
- 이진 범주형은 0/1로 인코딩 (Label Encoding)

**핵심 개념**:
- 머신러닝 모델은 숫자만 처리 가능
- 이진 변수: Label Encoding (0/1)
- 다중 범주: One-Hot Encoding 권장

**대안**:
```python
# sklearn LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['sex'] = le.fit_transform(df['sex'])

# replace 사용
df['sex'] = df['sex'].replace({'male': 0, 'female': 1})

# 조건문 사용
df['sex'] = (df['sex'] == 'female').astype(int)
```

**흔한 실수**:
- 딕셔너리 키 오타 ('Male' vs 'male')
- 인코딩 순서 혼동 (어떤 값이 0/1인지)

**실무 팁**:
- 인코딩 매핑을 문서화하여 추후 해석에 활용
- 여성=1로 설정하면 계수가 양수일 때 생존 확률 증가 해석이 직관적

---

### Q5. 데이터 분할 ⭐⭐

**문제**: X와 y로 분리한 후, 70:30 비율로 훈련/테스트 세트를 나누세요.

In [ ]:
# 전처리된 데이터 준비
df = titanic[['pclass', 'sex', 'age', 'fare', 'survived']].copy()
df['age'] = df['age'].fillna(df['age'].median())
df['sex'] = df['sex'].map({'male': 0, 'female': 1})

X = df.drop(columns=['survived'])
y = df['survived']

# 정답 코드: 70:30 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,       # 30%를 테스트로
    random_state=42,
    stratify=y           # 클래스 비율 유지
)

print(f"훈련 세트: {len(X_train)}개 ({len(X_train)/len(X)*100:.0f}%)")
print(f"테스트 세트: {len(X_test)}개 ({len(X_test)/len(X)*100:.0f}%)")

In [ ]:
# 테스트
assert len(X_train) + len(X_test) == len(X), "분할 후 총 개수가 맞아야 합니다"
assert abs(len(X_test) / len(X) - 0.3) < 0.01, "30% 비율이어야 합니다"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 특성(X)과 타겟(y) 분리
2. `train_test_split()`으로 분할
3. `test_size=0.3`으로 30% 테스트 비율 지정

**핵심 개념**:
- 훈련 세트: 모델 학습용
- 테스트 세트: 성능 평가용 (학습에 사용 금지)
- `stratify=y`: 원본과 동일한 클래스 비율 유지

**대안**:
```python
# stratify 없이 (랜덤 분할)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# shuffle=False (시계열 데이터)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, shuffle=False
)
```

**흔한 실수**:
- random_state 미지정 (재현 불가)
- stratify 누락 (불균형 데이터에서 편향)

**실무 팁**:
- 일반적인 비율: 80:20 또는 70:30
- 데이터가 적으면 교차 검증(Cross-Validation) 권장
- random_state는 팀 내 동일하게 유지

---

### Q6. 모델 훈련 ⭐⭐⭐

**문제**: LogisticRegression 모델을 생성하고 훈련한 후, 훈련 세트에서의 정확도를 출력하세요.

In [ ]:
# 정답 코드
# 모델 생성
model = LogisticRegression(max_iter=1000, random_state=42)

# 모델 훈련
model.fit(X_train, y_train)

# 훈련 세트 정확도
train_accuracy = model.score(X_train, y_train)
print(f"훈련 세트 정확도: {train_accuracy:.3f}")

In [ ]:
# 테스트
assert train_accuracy > 0.7, "정확도가 70% 이상이어야 합니다"
assert hasattr(model, 'coef_'), "모델이 훈련되어야 합니다"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. `LogisticRegression()` 객체 생성
2. `fit(X_train, y_train)`으로 훈련
3. `score()`로 정확도 확인

**핵심 개념**:
- `max_iter`: 최대 반복 횟수 (수렴 보장)
- `random_state`: 재현성 확보
- `score()`: 정확도 반환 (분류 모델)

**대안**:
```python
# accuracy_score 사용
from sklearn.metrics import accuracy_score
y_train_pred = model.predict(X_train)
accuracy = accuracy_score(y_train, y_train_pred)

# 정규화 강도 조절
model = LogisticRegression(C=0.1)  # C가 작을수록 강한 정규화
```

**흔한 실수**:
- fit() 호출 누락
- 테스트 데이터로 훈련 (데이터 누수)

**실무 팁**:
- 훈련 정확도가 테스트 정확도보다 훨씬 높으면 과적합 의심
- 훈련 정확도가 낮으면 모델이 너무 단순하거나 특성이 부족

---

### Q7. 예측 수행 ⭐⭐⭐

**문제**: 테스트 세트에 대해 클래스 예측과 확률 예측을 수행하세요.

In [ ]:
# 정답 코드
# 클래스 예측 (0 또는 1)
y_pred = model.predict(X_test)

# 확률 예측
y_prob = model.predict_proba(X_test)

print("클래스 예측 (처음 5개):")
print(y_pred[:5])

print("\n확률 예측 (처음 5개):")
print("[사망 확률, 생존 확률]")
print(y_prob[:5].round(3))

In [ ]:
# 테스트
assert len(y_pred) == len(X_test), "예측 개수가 맞아야 합니다"
assert y_prob.shape == (len(X_test), 2), "확률은 2개 클래스여야 합니다"
assert np.allclose(y_prob.sum(axis=1), 1), "확률 합은 1이어야 합니다"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `predict()`: 최종 클래스 예측 (0 또는 1)
- `predict_proba()`: 각 클래스의 확률 반환

**핵심 개념**:
- 확률 > 0.5 이면 클래스 1로 예측 (기본 임계값)
- `y_prob[:, 0]`: 사망 확률
- `y_prob[:, 1]`: 생존 확률

**대안**:
```python
# 생존 확률만 추출
survival_prob = model.predict_proba(X_test)[:, 1]

# 임계값 변경 (0.3)
y_pred_custom = (survival_prob > 0.3).astype(int)
```

**흔한 실수**:
- `predict_proba`와 `predict` 혼동
- 확률 인덱스 혼동 ([:, 0] vs [:, 1])

**실무 팁**:
- 임계값 조정으로 정밀도/재현율 트레이드오프 조절
- 확률값은 모델 신뢰도 평가에 활용

---

### Q8. 성능 평가 ⭐⭐⭐⭐

**문제**: 테스트 세트에서 정확도, 정밀도, 재현율, F1 스코어를 모두 계산하세요.

In [ ]:
# 정답 코드
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("=" * 40)
print("테스트 세트 성능 평가")
print("=" * 40)
print(f"정확도 (Accuracy):  {accuracy:.3f}")
print(f"정밀도 (Precision): {precision:.3f}")
print(f"재현율 (Recall):    {recall:.3f}")
print(f"F1 스코어:          {f1:.3f}")

In [ ]:
# 테스트
assert 0 <= accuracy <= 1, "정확도는 0~1 사이"
assert 0 <= precision <= 1, "정밀도는 0~1 사이"
assert 0 <= recall <= 1, "재현율은 0~1 사이"
assert 0 <= f1 <= 1, "F1은 0~1 사이"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- sklearn.metrics에서 각 평가 함수 import
- 실제값(y_test)과 예측값(y_pred) 비교

**핵심 개념**:
| 지표 | 계산식 | 의미 |
|------|--------|------|
| 정확도 | (TP+TN) / 전체 | 전체 정답 비율 |
| 정밀도 | TP / (TP+FP) | 양성 예측의 정확도 |
| 재현율 | TP / (TP+FN) | 실제 양성 탐지율 |
| F1 | 2*P*R / (P+R) | 정밀도/재현율 조화평균 |

**대안**:
```python
# classification_report로 한번에
print(classification_report(y_test, y_pred))

# 딕셔너리로 저장
metrics = {
    'accuracy': accuracy_score(y_test, y_pred),
    'precision': precision_score(y_test, y_pred),
    'recall': recall_score(y_test, y_pred),
    'f1': f1_score(y_test, y_pred)
}
```

**흔한 실수**:
- y_test와 y_pred 순서 혼동
- 다중 클래스에서 average 파라미터 누락

**실무 팁**:
- 불균형 데이터: 정확도보다 F1 또는 재현율 중시
- 비용 비대칭: FP/FN 비용에 따라 정밀도 또는 재현율 선택

---

### Q9. 혼동 행렬 시각화 ⭐⭐⭐⭐

**문제**: 테스트 세트의 혼동 행렬을 계산하고 Plotly로 시각화하세요.

In [ ]:
# 정답 코드
# 혼동 행렬 계산
cm = confusion_matrix(y_test, y_pred)

print("혼동 행렬:")
print(cm)

# Plotly 시각화
fig = px.imshow(
    cm,
    labels=dict(x="예측", y="실제", color="건수"),
    x=['사망 (0)', '생존 (1)'],
    y=['사망 (0)', '생존 (1)'],
    text_auto=True,
    color_continuous_scale='Blues',
    title='혼동 행렬 (Confusion Matrix)'
)
fig.show()

In [ ]:
# 테스트
assert cm.shape == (2, 2), "2x2 행렬이어야 합니다"
assert cm.sum() == len(y_test), "합이 테스트 개수와 같아야 합니다"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. `confusion_matrix()`로 혼동 행렬 계산
2. `px.imshow()`로 히트맵 시각화
3. 축 레이블 추가로 해석 용이하게

**핵심 개념**:
```
              예측 0    예측 1
실제 0 (사망)   TN        FP
실제 1 (생존)   FN        TP
```
- TN: True Negative (정확히 사망 예측)
- FP: False Positive (사망인데 생존 예측)
- FN: False Negative (생존인데 사망 예측)
- TP: True Positive (정확히 생존 예측)

**대안**:
```python
# 정규화된 혼동 행렬 (비율)
cm_normalized = confusion_matrix(y_test, y_pred, normalize='true')

# seaborn 사용 (matplotlib 기반)
import seaborn as sns
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
```

**흔한 실수**:
- 축 레이블 순서 혼동 (x=예측, y=실제)
- normalize 옵션 미인지

**실무 팁**:
- FP/FN의 비즈니스 비용이 다르면 임계값 조정
- 정규화 버전은 클래스 불균형 시 유용

---

### Q10. 종합 파이프라인 ⭐⭐⭐⭐⭐

**문제**: 새로운 승객 정보가 주어졌을 때, 생존 확률을 예측하는 함수를 작성하세요.

In [ ]:
# 정답 코드
def predict_survival(pclass, sex, age, fare):
    """
    승객 정보를 받아 생존 확률을 예측합니다.
    
    Parameters:
    - pclass: 객실 등급 (1, 2, 3)
    - sex: 성별 ('male' 또는 'female')
    - age: 나이
    - fare: 운임
    
    Returns:
    - 생존 확률 (0~1)
    """
    # 성별 인코딩
    sex_encoded = 1 if sex == 'female' else 0
    
    # 입력 데이터 구성 (모델 훈련 시 사용한 특성 순서와 동일해야 함)
    input_data = pd.DataFrame({
        'pclass': [pclass],
        'sex': [sex_encoded],
        'age': [age],
        'fare': [fare]
    })
    
    # 생존 확률 예측 (클래스 1의 확률)
    survival_prob = model.predict_proba(input_data)[0, 1]
    
    return survival_prob

# 테스트 케이스
print("=== 생존 확률 예측 ===")

# 1등석 여성
prob1 = predict_survival(pclass=1, sex='female', age=25, fare=100)
print(f"1등석, 여성, 25세: {prob1:.1%}")

# 3등석 남성
prob2 = predict_survival(pclass=3, sex='male', age=30, fare=10)
print(f"3등석, 남성, 30세: {prob2:.1%}")

# 2등석 어린이
prob3 = predict_survival(pclass=2, sex='male', age=5, fare=30)
print(f"2등석, 남자 어린이, 5세: {prob3:.1%}")

In [ ]:
# 테스트
assert 0 <= predict_survival(1, 'female', 25, 100) <= 1, "확률은 0~1 사이"
assert predict_survival(1, 'female', 25, 100) > predict_survival(3, 'male', 30, 10), "1등석 여성이 3등석 남성보다 높아야 함"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 함수 정의 (입력: 원시 데이터, 출력: 확률)
2. 범주형 변수 인코딩 (sex)
3. DataFrame으로 변환 (컬럼명 일치 필수)
4. `predict_proba()`로 확률 예측

**핵심 개념**:
- **입력 변환**: 사용자 친화적 입력 -> 모델 입력 형식
- **특성 순서**: 훈련 시와 동일한 컬럼 순서 유지
- **확률 반환**: 클래스 1(생존)의 확률만 추출

**대안**:
```python
# 여러 승객 동시 예측
def predict_survival_batch(passengers_df):
    passengers_df['sex'] = passengers_df['sex'].map({'male': 0, 'female': 1})
    return model.predict_proba(passengers_df)[:, 1]

# 클래스와 확률 모두 반환
def predict_full(pclass, sex, age, fare):
    prob = predict_survival(pclass, sex, age, fare)
    label = '생존' if prob >= 0.5 else '사망'
    return {'prediction': label, 'probability': prob}
```

**흔한 실수**:
- 컬럼 순서 불일치
- 인코딩 매핑 오류 (훈련 시와 다른 매핑)
- 2D 배열 반환 처리 안 함 ([0, 1] 인덱싱)

**실무 팁**:
- 예측 함수는 API 또는 웹 서비스의 핵심 로직
- 입력 검증 로직 추가 권장 (범위 체크, 타입 검증)
- 로깅으로 예측 기록 남기기

---

## 📊 퀴즈 요약

| 번호 | 난이도 | 주제 | 핵심 함수 |
|------|--------|------|----------|
| Q1 | ⭐ | 데이터 탐색 | `value_counts()` |
| Q2 | ⭐ | 그룹별 집계 | `groupby().mean()` |
| Q3 | ⭐⭐ | 결측치 처리 | `fillna()` |
| Q4 | ⭐⭐ | 범주형 인코딩 | `map()` |
| Q5 | ⭐⭐ | 데이터 분할 | `train_test_split()` |
| Q6 | ⭐⭐⭐ | 모델 훈련 | `LogisticRegression().fit()` |
| Q7 | ⭐⭐⭐ | 예측 수행 | `predict()`, `predict_proba()` |
| Q8 | ⭐⭐⭐⭐ | 성능 평가 | `accuracy_score`, `f1_score` |
| Q9 | ⭐⭐⭐⭐ | 혼동 행렬 | `confusion_matrix()`, `px.imshow()` |
| Q10 | ⭐⭐⭐⭐⭐ | 종합 파이프라인 | 전체 통합 |